In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.stem.porter import PorterStemmer

In [14]:
df=pd.read_csv("data/text_emotion.csv")

In [15]:
df.head()

,text,emotion
0,carefully word blog posts amount criticism hea...,0
1,cannot remember little mermaid feeling carefre...,1
2,not feeling super well turns cold knocked next...,1
3,feel honored part group amazing talents,1
4,think helping also began feel pretty lonely lo...,0


In [16]:
df['emotion'].value_counts()

emotion
1    134205
0    120334
Name: count, dtype: int64

In [17]:
# balanced data set

In [18]:
df.shape

(254539, 2)

In [19]:
X=df[['text']]
y=df['emotion']

In [20]:
#train_test split

from sklearn.model_selection import train_test_split

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [22]:
# Preprocessing

In [23]:
import re
def remove_special(text):
    clean_text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    return clean_text

In [24]:

def remove_html(text):
    pattern=re.compile('<.*?>')
    return pattern.sub(r'', text)

In [25]:
def remove_extra_white_spaces(text):
    pattern=r'\s+[a-zA-Z]\s+'
    without_space=re.sub(pattern=pattern,repl=" ",string=text)
    return without_space
    

In [26]:
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

In [27]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sapta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:



def remove_stop(text):
    new_text=[]
    for word in text.split():
        if word in stopwords:
            new_text.append('')
        else:
            new_text.append(word)
    x=new_text[:]
    new_text.clear()
    
    return " ".join(x)

In [30]:
X_train=X_train.reset_index()
X_test=X_test.reset_index()

In [31]:
X_train=X_train.drop(columns=['index'],axis=1)
X_test=X_test.drop(columns=['index'],axis=1)

In [32]:
X_train.head()

,text
0,feelings victimized
1,sat chair various times said anything times al...
2,still challenging feel reassured description son
3,feel sorry people openly slander pass ill judg...
4,love earlier master presence could feel lot ca...


In [33]:
def clean_text(X_train):
    X_train['text']=X_train['text'].str.lower()
    X_train['text']=X_train['text'].apply(remove_special)
    X_train['text']=X_train['text'].apply(remove_html)
    X_train['text']=X_train['text'].apply(remove_extra_white_spaces)
    X_train['text']=X_train['text'].apply(remove_stop)
    X_train['text']=X_train['text'].apply(stem_words)

    return X_train

In [34]:
X_train=clean_text(X_train)
X_test=clean_text(X_test)

In [35]:
X_train.head()

,text
0,feel victim
1,sat chair variou time said anyth time alway wa...
2,still challeng feel reassur descript son
3,feel sorri peopl openli slander pass ill judge...
4,love earlier master presenc could feel lot cas...


In [9]:
class Stemmer(object):
                def __init__(self):
                  self.ps = PorterStemmer()
                def __call__(self, text):
                  return [self.ps.stem(word) for word in word_tokenize(text)]

In [10]:
text_pipeline=Pipeline(steps=['vectorizer',TfidfVectorizer(analyzer='word',tokenizer=PorterStemmer,max_features=2000)])


In [11]:
preprocessor=ColumnTransformer([
                    ('text_pipeline',text_pipeline,['text'])
                    
            ])

In [36]:
X_train

,text
0,feel victim
1,sat chair variou time said anyth time alway wa...
2,still challeng feel reassur descript son
3,feel sorri peopl openli slander pass ill judge...
4,love earlier master presenc could feel lot cas...
...,...
203626,felt need share one sentenc found one book del...
203627,posit idea sat feel groggi wait someth chang h...
203628,feel quit jolli even though pretti tire
203629,wear shirt feel artist artist look artist ye son


In [38]:
input_feature_train_arr=preprocessor.fit_transform(X_train).toarray()
input_feature_test_arr=preprocessor.transform(X_test).toarray()

TypeError: 'TfidfVectorizer' object is not subscriptable